In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_638064/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_10_48_9,0.999999,0.688936,0.622607,0.999999,3.365138e-07,0.184661,0.264372,5.795695e-07,0.000207,0.000580,1.000000,0.000585,1615.809254,3889.764073,"Hidden Size=[36, 18], regularizer=0.02, learni..."
1,model_10_48_8,0.999999,0.688944,0.622612,0.999999,3.385642e-07,0.184656,0.264368,5.714949e-07,0.000207,0.000582,1.000000,0.000586,1615.797104,3889.751924,"Hidden Size=[36, 18], regularizer=0.02, learni..."
2,model_10_48_7,0.999999,0.689005,0.622654,0.999999,3.411129e-07,0.184620,0.264339,5.509715e-07,0.000211,0.000584,1.000000,0.000589,1615.782104,3889.736924,"Hidden Size=[36, 18], regularizer=0.02, learni..."
3,model_10_48_6,0.999999,0.688987,0.622635,0.999999,3.469587e-07,0.184630,0.264352,5.379367e-07,0.000211,0.000589,1.000000,0.000594,1615.748120,3889.702940,"Hidden Size=[36, 18], regularizer=0.02, learni..."
4,model_9_36_2,0.999999,0.692210,0.629742,0.999999,3.471571e-07,0.182718,0.259374,7.876208e-07,0.000247,0.000589,1.000000,0.000594,799.746977,1903.747740,"Hidden Size=[24, 12], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,model_5_0_0,0.038633,-0.145577,-0.188185,-0.282886,4.118976e-01,0.680064,0.832349,5.301958e-01,0.361875,0.641792,8.751017,0.646787,291.773961,707.566456,"Hidden Size=[36], regularizer=0.02, learning_r..."
4980,model_6_0_0,0.025293,-0.026455,-0.106554,0.006914,4.176135e-01,0.609348,0.775164,3.606314e-01,0.344895,0.646230,-0.209012,0.651259,51.746398,123.434759,"Hidden Size=[4, 2], regularizer=0.02, learning..."
4981,model_8_0_1,-0.001632,-0.405704,-0.412535,0.004465,4.291494e-01,0.834486,0.989510,4.418324e-01,0.356835,0.655095,-15.151319,0.660193,243.691900,590.663569,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
4982,model_8_0_0,-0.151847,-0.167238,-0.310897,-0.174582,4.935091e-01,0.692923,0.918311,5.212962e-01,0.376823,0.702502,-17.573537,0.707969,243.412428,590.384097,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
